<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-21 12:26:49
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: -9.10 K (-0.07%)
Current PnL: -17.01 L (-11.57%)
CY Booked + Current PnL: -8.28 L (-5.63%)
-------------------
Total profit:  3.78 L
Total loss:  -20.80 L
-------------------
Total Booked + Current PnL: 18.70 L (15.58%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.31%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.13 L (55.04%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 9.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,7098.95,8285.00,-12.53,H-LC,7.33,174702.0,18525.0,7565.0,0.74,11.86,4.33,16.71,22.0,2.45,1.29,31.37,X40N,BTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.02,112752.0,8440.0,8208.0,0.55,8.09,7.28,15.96,9.0,1.03,0.84,26.36,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.91,219976.0,18496.0,8711.0,0.14,9.18,3.96,13.51,60.0,2.12,1.63,22.93,XY25,NTT,BANKS
37,HINDUNILVR,2413.81,2723.00,-5.76,H-LC,6.96,296150.0,25803.0,8825.0,-0.96,9.54,2.98,12.81,24.0,2.92,2.19,23.86,XY25,NTT,FMCG
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,160962.0,34762.0,11412.0,0.70,27.55,7.09,36.59,108.0,3.05,1.19,43.51,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.37,178992.0,-816.0,113230.0,0.04,-0.45,63.26,62.52,88.0,-0.01,1.33,33.48,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-30.68,H-MC,5.74,200595.0,-225.0,70349.0,-0.48,-0.11,35.07,34.92,80.0,-0.00,1.49,4.56,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.64,101320.0,816.0,29818.0,-0.55,0.81,29.43,30.49,79.0,0.03,0.75,33.48,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,3.06,147802.0,-2130.0,16731.0,-0.75,-1.42,11.32,9.74,4.0,-0.13,1.10,4.73,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.44,H-LC,3.59,214720.0,10220.0,31199.0,1.37,5.00,14.53,20.25,10.0,0.33,1.59,14.17,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.49,H-LC,3.71,187416.0,-12894.0,72867.0,0.49,-6.44,38.88,29.94,16.0,-0.18,1.39,29.48,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.26,H-LC,3.73,275980.0,10554.0,46227.0,-0.92,3.98,16.75,21.40,11.0,0.23,2.05,12.07,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.58,H-LC,5.22,174332.0,-27054.0,114676.0,-0.25,-13.43,65.78,43.51,27.0,-0.24,1.29,21.35,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,88013.0,-25536.0,65587.0,15.26,-22.49,74.52,35.27,268.0,-0.39,0.65,115.41,XR,NTT,HOTELS
60,RAJESHEXPO,517.65,518.00,1747.22,L-SC,7.69,50406.0,-86771.0,86865.0,2.86,-63.25,172.33,0.07,267.0,-1.00,0.37,25.36,OX40N,NTT,JEWELLERY
66,SAMMAANCAP,170.35,326.00,-164.77,M-SC,7.52,74766.0,-27444.0,120837.0,2.59,-26.85,161.62,91.37,208.0,-0.23,0.55,21.37,XY25,NTT,FINANCE
75,SURYODAY,189.78,240.00,46.93,H-SC,5.32,103719.0,-46207.0,85879.0,2.55,-30.82,82.80,26.46,135.0,-0.54,0.77,32.87,XR,NTT,BANKS
31,HAPPSTMNDS,919.80,1488.71,-10.29,H-SC,8.82,63514.0,-33065.0,92800.0,2.22,-34.24,146.11,61.85,132.0,-0.36,0.47,11.99,AR,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-4.94,M-SC,26.61,91530.0,-85285.0,166081.0,-2.90,-48.23,181.45,45.70,197.0,-0.51,0.68,6.47,XY24,NTT,TRAVEL
25,DABUR,505.20,735.00,-6.18,H-MC,3.81,204360.0,7332.0,82296.0,-2.06,3.72,40.27,45.49,102.0,0.09,1.51,17.25,XY24,BTT,FMCG
14,BERGEPAINT,561.33,680.00,-19.75,H-MC,9.04,215723.0,-11616.0,59669.0,-2.00,-5.11,27.66,21.14,106.0,-0.19,1.60,21.59,XY24,NTT,PAINTS
15,BLUESTARCO,1646.70,2326.38,9.48,H-MC,5.66,189000.0,24330.0,43640.0,-1.94,14.78,23.09,41.28,89.0,0.56,1.40,23.94,X40N,ATH,AC
43,INDIGOPNTS,1407.73,1408.00,115.96,M-SC,2.02,141794.0,-32765.0,32797.0,-1.52,-18.77,23.13,0.02,221.0,-1.00,1.05,22.96,OX40N,NTT,PAINTS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,100.21,M-SC,9.36,86224.0,-14553.0,14649.0,0.56,-14.44,16.99,0.09,245.0,-0.99,0.64,13.28,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,87.86,H-SC,16.24,118908.0,-23571.0,43092.0,0.82,-16.54,36.24,13.70,163.0,-0.55,0.88,40.34,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,115.96,M-SC,2.02,141794.0,-32765.0,32797.0,-1.52,-18.77,23.13,0.02,221.0,-1.00,1.05,22.96,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-18.38,H-SC,3.11,116316.0,-27487.0,54238.0,-0.58,-19.11,46.63,18.60,149.0,-0.51,0.86,28.36,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.52,H-SC,2.91,217260.0,-52407.0,88751.0,-1.15,-19.43,40.85,13.47,138.0,-0.59,1.61,9.89,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-26.44,H-LC,16.18,242783.0,-53151.0,147054.0,0.45,-17.96,60.57,31.73,1.0,-0.36,1.80,3.65,X40,BTT,IT
45,INFY,1461.46,2275.00,-17.80,H-LC,11.93,268625.0,7024.0,138610.0,0.30,2.68,51.60,55.67,3.0,0.05,1.99,8.95,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,3.06,147802.0,-2130.0,16731.0,-0.75,-1.42,11.32,9.74,4.0,-0.13,1.10,4.73,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.45,H-LC,9.29,277904.0,12864.0,83427.0,-0.21,4.85,30.02,36.33,5.0,0.15,2.06,18.80,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.03,H-LC,11.93,200265.0,-16254.0,109164.0,0.26,-7.51,54.51,42.91,7.0,-0.15,1.48,4.88,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.80,H-MC,14.36,160962.0,34762.0,11412.0,0.70,27.55,7.09,36.59,108.0,3.05,1.19,43.51,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.93,H-LC,8.70,229986.0,45505.0,24861.0,-0.94,24.67,10.81,38.14,38.0,1.83,1.70,37.89,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,160962.0,34762.0,11412.0,0.70,27.55,7.09,36.59,108.0,3.05,1.19,43.51,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,10.17,H-SC,11.17,131317.0,6020.0,52986.0,-0.09,4.80,40.35,47.09,141.0,0.11,0.97,64.35,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,9.48,H-MC,5.66,189000.0,24330.0,43640.0,-1.94,14.78,23.09,41.28,89.0,0.56,1.40,23.94,X40N,ATH,AC
90,VOLTAS,1278.28,1918.49,-3.18,H-MC,7.38,203250.0,11508.0,84532.0,-1.08,6.00,41.59,50.08,99.0,0.14,1.51,13.21,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,11.35,177846.0,26933.0,30376.0,-1.06,17.85,17.08,37.97,93.0,0.89,1.32,47.67,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,11.35,177846.0,26933.0,30376.0,-1.06,17.85,17.08,37.97,93.0,0.89,1.32,47.67,AR,ATH,AUTO
61,RAJOOENG,102.50,143.33,-28.76,H-SC,8.70,107990.0,5490.0,35345.0,0.11,5.36,32.73,39.83,114.0,0.16,0.80,6.92,AR,ATH,MISC
42,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.09,140911.0,17575.0,116181.0,1.34,14.25,82.45,108.45,119.0,0.15,1.04,39.79,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,10.17,H-SC,11.17,131317.0,6020.0,52986.0,-0.09,4.80,40.35,47.09,141.0,0.11,0.97,64.35,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-41.55,M-SC,5.25,98768.0,7270.0,71478.0,1.45,7.94,72.37,86.07,223.0,0.10,0.73,42.88,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,7100.00,L-SC,7.09,79006.0,-14804.0,91418.0,-1.29,-15.78,115.71,81.67,269.0,-0.16,0.59,53.00,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,88013.0,-25536.0,65587.0,15.26,-22.49,74.52,35.27,268.0,-0.39,0.65,115.41,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-0.30,H-SC,15.73,92867.0,-8096.0,102627.0,0.89,-8.02,110.51,93.63,148.0,-0.08,0.69,38.32,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-17.34,H-SC,12.36,93825.0,-4155.0,26027.0,-0.41,-4.24,27.74,22.32,152.0,-0.16,0.70,36.08,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-9.59,H-LC,6.92,97069.0,-676.0,38070.0,1.15,-0.69,39.22,38.26,35.0,-0.02,0.72,27.11,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,88013.0,-25536.0,65587.0,15.26,-22.49,74.52,35.27,268.0,-0.39,0.65,115.41,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,11.35,177846.0,26933.0,30376.0,-1.06,17.85,17.08,37.97,93.0,0.89,1.32,47.67,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.93,H-LC,8.70,229986.0,45505.0,24861.0,-0.94,24.67,10.81,38.14,38.0,1.83,1.70,37.89,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.53,H-LC,7.33,174702.0,18525.0,7565.0,0.74,11.86,4.33,16.71,22.0,2.45,1.29,31.37,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.58,H-LC,5.22,174332.0,-27054.0,114676.0,-0.25,-13.43,65.78,43.51,27.0,-0.24,1.29,21.35,X40,BTT,PAINTS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.76
1,25,41.32
2,50,70.91


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.54
LC    36.15
MC    26.27
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.45
X40      14.38
X40N     13.83
XY25     11.84
XR        9.11
AR        8.44
OX40N     7.44
X5K       2.23
MH        1.72
X200      1.39
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.92
H-MC    23.37
H-SC    22.68
M-SC    13.25
M-LC     7.19
M-MC     2.58
L-SC     1.61
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.56,1.03,29.28
IT,11.90,-16.24,74.29
FINANCE,8.07,-19.64,63.90
BANKS,7.59,-11.21,58.44
MISC,5.87,-16.09,82.64
PAINTS,5.55,-16.53,39.50
ELECTRICAL,4.71,-10.18,57.90
AUTO,4.56,-9.82,57.60
AC,3.64,1.29,32.14


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2794675.0,23
AR,1067226.0,10
XR,1065089.0,13
X40,796830.0,11
X40N,499587.0,11
OX40N,454809.0,10
XY25,448939.0,8
SR,191458.0,2
X5K,138360.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2711860.0,27
M-SC,1458711.0,17
H-MC,1225714.0,18
H-LC,1132210.0,19
M-LC,408338.0,5
M-MC,329515.0,2
L-SC,243870.0,3
L-MC,58784.0,1
L-LC,43642.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      919042.0      6
M-SC       XY24      858750.0      7
H-SC       AR        650824.0      5
           XR        512850.0      6
H-LC       X40       509701.0      6
H-MC       XY24      478271.0      5
M-MC       XY24      329515.0      2
H-LC       AR        228258.0      2
H-MC       X40       206200.0      4
H-SC       OX40N     197613.0      4
H-MC       XY25      195193.0      2
H-SC       SR        191458.0      2
H-LC       X40N      188585.0      5
H-SC       X40N      187087.0      3
M-LC       XY24      165062.0      2
M-SC       AR        157768.0      2
L-SC       XR        157005.0      2
M-LC       X5K       130152.0      1
H-MC       X40N      123915.0      3
M-SC       OX40N     121620.0      4
           XY25      120837.0      1
           XR        118807.0      2
H-MC       XR        113230.0      1
M-LC       XR        104413.0      1
L-SC       OX40N      86865.0      1
M-SC       X40        80929.0      1
H-LC       XY25       80556.0      3
           X200       72867.0      1
L-MC       XR         58784.0      1
H-SC       MH         52986.0      1
H-MC       OX40N      48711.0      1
H-LC       XY24       44035.0      1
L-LC       XY25       43642.0      1
H-MC       AR         30376.0      1
           MH         29818.0      1
M-LC       XY25        8711.0      1
H-LC       X5K         8208.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
